# Machine Learning

import libraries

In [143]:
import os  
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
# --------cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
# -------- classification
import sklearn
from sklearn import neighbors, tree, ensemble, naive_bayes, svm
# *** KNN
from sklearn.neighbors import KNeighborsClassifier
# *** Decision Tree; Random Forest
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# *** Naive Bayes
from sklearn.naive_bayes import GaussianNB
# *** SVM classifier
from sklearn.svm import SVC
# --------  metrics:
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import make_scorer
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler


read "clean_data" file that include the data after edited

In [144]:
#Load the CSV into a DataFrame
df = pd.read_csv(r'clean_data.csv')

#A copy of the data for backup
clean_df = df.iloc[:,1:].copy()

Editing the data for machine learning

In [145]:
df_ml = clean_df.copy()
df_ml.drop(['Game'],axis=1, inplace = True)
df_ml

,Pos,Genre,Console,Publisher,Developer,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Total_Sales,Relese_Month,Relese_Year,Hit_Games
0,1,Action,Series,Sony Computer Entertainment,SIE Santa Monica Studio,0,0,0,0,51000000,3,2005,1
1,2,Action,Series,KOEI,Omega Force,0,0,0,0,47820000,6,1997,1
2,3,Action,Series,Capcom,Capcom,0,0,0,0,25200000,10,2001,1
3,4,Action,Series,KOEI,Omega Force,0,0,0,0,21150000,6,1997,1
4,5,Action,Series,Konami,Konami,0,0,0,0,20000000,10,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17505,461,Visual+Novel,PSP,Idea Factory,Idea Factory,0,0,30000,0,30000,11,2012,0
17506,462,Visual+Novel,PSV,5pb,5pb. Games,0,0,30000,0,30000,2,2014,0
17507,463,Visual+Novel,PS3,Nippon Ichi Software,Nippon Ichi Software,0,0,40000,0,40000,8,2014,0
17508,464,Visual+Novel,PS4,PQube,Kadokawa Games,0,0,30000,0,30000,11,2016,0


In [146]:
df_ml['Genre'] = preprocessing.LabelEncoder().fit_transform(clean_df['Genre'])
df_ml['Console'] = preprocessing.LabelEncoder().fit_transform(clean_df['Console'])
df_ml['Publisher'] = preprocessing.LabelEncoder().fit_transform(clean_df['Publisher'])
df_ml['Developer'] = preprocessing.LabelEncoder().fit_transform(clean_df['Developer'])
df_ml

,Pos,Genre,Console,Publisher,Developer,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Total_Sales,Relese_Month,Relese_Year,Hit_Games
0,1,0,36,848,2483,0,0,0,0,51000000,3,2005,1
1,2,0,36,485,2075,0,0,0,0,47820000,6,1997,1
2,3,0,36,165,524,0,0,0,0,25200000,10,2001,1
3,4,0,36,485,2075,0,0,0,0,21150000,6,1997,1
4,5,0,36,510,1582,0,0,0,0,20000000,10,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17505,461,17,31,443,1379,0,0,30000,0,30000,11,2012,0
17506,462,17,32,16,53,0,0,30000,0,30000,2,2014,0
17507,463,17,27,663,2017,0,0,40000,0,40000,8,2014,0
17508,464,17,28,698,1534,0,0,30000,0,30000,11,2016,0


Auxiliary function for splitting the data for machine learning

In [147]:
def split_to_train_and_test(dataset, label_column, test_ratio, rand_state):
    training = dataset.columns[dataset.columns != label_column]
    target = label_column
    X = dataset[training]
    y = dataset[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    return X_train, X_test, y_train, y_test

test_ratio, rand_state = 0.2, 42
X_train, X_test, y_train, y_test = split_to_train_and_test(df_ml, "Hit_Games", test_ratio, rand_state)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(14008, 12) (3502, 12) (14008,) (3502,)


Scale the data

In [148]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Auxiliary function for finding the best k for the knn algorithm

In [149]:
def find_best_k_for_KNN(X_train, y_train):
    params={"n_neighbors":[3,7,9,11]}
    knn = KNeighborsClassifier()
    clf = GridSearchCV(knn, params,scoring=make_scorer(metrics.f1_score, greater_is_better=True))
    clf.fit(X_train,y_train)
    best_K = clf.best_params_['n_neighbors']
    best_f1_val = clf.best_score_
    return best_K, best_f1_val
best_K, best_f1_KNN_params = find_best_k_for_KNN(X_train, y_train)
# --- add additional code to check your code if needed:
best_K, best_f1_KNN_params

(3, 0.7874169541147212)

Auxiliary function for finding the best depth in a tree and a minimum amount of values per group

In [150]:
def find_best_decision_tree_params(X_train, y_train):
    parameters={'max_depth':[2,4,6,8,10],'min_samples_split':[5,10,15,20]}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters,scoring=make_scorer(metrics.f1_score, greater_is_better=True))
    clf.fit(X_train,y_train)
    best_max_depth = clf.best_params_['max_depth']
    best_min_samples_split = clf.best_params_['min_samples_split']
    best_f1_val = clf.best_score_
    return best_max_depth, best_min_samples_split, best_f1_val
best_max_dep, best_min_smpl_splt, best_f1_DT_params = find_best_decision_tree_params(X_train, y_train)
best_max_dep, best_min_smpl_splt, best_f1_DT_params

(2, 5, 1.0)

Auxiliary function that returns the model of the requested algorithm

In [151]:
def get_classifier_obj(classifier_name, params):

    if(classifier_name == 'KNN'):
        clf=KNeighborsClassifier(n_neighbors=params['n_neighbors'])
    if(classifier_name == 'decision_tree'):
        clf=tree.DecisionTreeClassifier(max_depth=params['max_depth'], min_samples_split=params['min_samples_split'])
    if(classifier_name == 'svm'):
        clf=svm.SVC()
    if(classifier_name == 'naive_bayes'):
        clf=GaussianNB()
    if(classifier_name == 'LogisticRegression'):    
        clf=linear_model.LogisticRegression()
    return clf

Auxiliary function for calculating the prediction results using the cross validation method

In [152]:
def calc_evaluation_val(eval_metric, y_true, y_predicted):
    if(eval_metric == 'accuracy'):
        evaluation_val = accuracy_score(y_true = y_true, y_pred = y_predicted)
    elif(eval_metric == 'precision'):
        evaluation_val=  precision_score(y_true = y_true, y_pred = y_predicted)
    elif(eval_metric == 'recall'):
        evaluation_val = recall_score(y_true = y_true, y_pred = y_predicted)
    elif(eval_metric == 'f1'):
        evaluation_val = f1_score(y_true = y_true, y_pred = y_predicted)
    elif(eval_metric == 'confusion_matrix'):
        evaluation_val = metrics.confusion_matrix(y_true = y_true, y_pred = y_predicted)
        df_confusion = pd.DataFrame(evaluation_val , index=['Actual Negative','Actual Positive'],columns=['Predicted Negative','Predicted Positive'])
        print("confusion matrix:\n")
        display(df_confusion)
        return
    return evaluation_val

Machine learning phase

In [153]:
params_knn = {'n_neighbors': best_K}
params_decision_tree = {'max_depth': best_max_dep, 'min_samples_split': best_min_smpl_splt}

In [154]:
clf_naive_bayes = get_classifier_obj("naive_bayes",None)
clf_svm = get_classifier_obj("svm",None)
clf_knn = get_classifier_obj("KNN",params_knn)   
clf_decision_tree = get_classifier_obj("decision_tree",params_decision_tree)
clf_logistic_regression = get_classifier_obj("LogisticRegression",None)

The training phase

In [155]:
clf_logistic_regression.fit(X_train,y_train)
clf_svm.fit(X_train,y_train)
clf_knn.fit(X_train,y_train)
clf_naive_bayes.fit(X_train,y_train)
clf_decision_tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=2, min_samples_split=5)

The prediction phase

In [156]:
train_logistic_regression = clf_logistic_regression.predict(X_train)
train_svm = clf_svm.predict(X_train)
train_knn = clf_knn.predict(X_train)
train_naive_bayes = clf_naive_bayes.predict(X_train)
train_decision_tree = clf_decision_tree.predict(X_train)

In [157]:
predicted_logistic_regression = clf_logistic_regression.predict(X_test)
predicted_svm = clf_svm.predict(X_test)
predicted_knn = clf_knn.predict(X_test)
predicted_naive_bayes = clf_naive_bayes.predict(X_test)
predicted_decision_tree = clf_decision_tree.predict(X_test)
#predicted_random_forest = trained_random_forest.predict(X_test)

Choosing the algorithm with the best results

In [158]:
train_scores = []
test_scores = []
predicted_train = {"logistic_regression": train_logistic_regression,
                  "naive_bayes": train_naive_bayes,
                  "svm": train_svm,
                  "knn": train_knn,
                  "decision_tree": train_decision_tree}
predicted_test = {"logistic_regression": predicted_logistic_regression,
                  "naive_bayes": predicted_naive_bayes,
                  "svm": predicted_svm,
                  "knn": predicted_knn,
                  "decision_tree": predicted_decision_tree}
evaluation = ["accuracy","precision","recall","f1"]

In [159]:
for alg in data_predicted.keys():
    print(f" \n\nthe algorithm is : {alg}")
    for eval in evaluation:
        train_result = calc_evaluation_val(eval, y_train, predicted_train[alg])
        train_scores.append(round(train_result, 5))
        print(eval,"on train data: ",train_result)
        test_result = calc_evaluation_val(eval, y_test, predicted_test[alg])
        test_scores.append(round(test_result, 5))
        print(eval,"on test data:  ",test_result ,"\n")
    #plotplot(evaluation,train_scores,test_scores,alg,'#33ffd6','salmon')
    calc_evaluation_val('confusion_matrix', y_test,data_predicted[alg])

 

the algorithm is : logistic_regression
accuracy on train data:  0.9670188463735009
accuracy on test data:   0.9663049685893775 

precision on train data:  0.9726487523992322
precision on test data:   0.9657142857142857 

recall on train data:  0.8334703947368421
recall on test data:   0.8352553542009885 

f1 on train data:  0.8976970770593447
f1 on test data:   0.8957597173144877 

confusion matrix:



,Predicted Negative,Predicted Positive
Actual Negative,2877,18
Actual Positive,100,507


 

the algorithm is : naive_bayes
accuracy on train data:  0.9444603083952028
accuracy on test data:   0.9483152484294689 

precision on train data:  0.816131498470948
precision on test data:   0.8297213622291022 

recall on train data:  0.8778782894736842
recall on test data:   0.8830313014827018 

f1 on train data:  0.845879556259905
f1 on test data:   0.8555466879489226 

confusion matrix:



,Predicted Negative,Predicted Positive
Actual Negative,2785,110
Actual Positive,71,536


 

the algorithm is : svm
accuracy on train data:  0.9803683609366076
accuracy on test data:   0.9751570531125071 

precision on train data:  0.9842837898518186
precision on test data:   0.983271375464684 

recall on train data:  0.9013157894736842
recall on test data:   0.871499176276771 

f1 on train data:  0.9409744580382057
f1 on test data:   0.9240174672489083 

confusion matrix:



,Predicted Negative,Predicted Positive
Actual Negative,2886,9
Actual Positive,78,529


 

the algorithm is : knn
accuracy on train data:  0.9653055396916048
accuracy on test data:   0.933466590519703 

precision on train data:  0.9336007130124777
precision on test data:   0.8450184501845018 

recall on train data:  0.8614309210526315
recall on test data:   0.7545304777594728 

f1 on train data:  0.8960650128314799
f1 on test data:   0.7972149695387294 

confusion matrix:



,Predicted Negative,Predicted Positive
Actual Negative,2811,84
Actual Positive,149,458


 

the algorithm is : decision_tree
accuracy on train data:  1.0
accuracy on test data:   1.0 

precision on train data:  1.0
precision on test data:   1.0 

recall on train data:  1.0
recall on test data:   1.0 

f1 on train data:  1.0
f1 on test data:   1.0 

confusion matrix:



,Predicted Negative,Predicted Positive
Actual Negative,2895,0
Actual Positive,0,607


# And the best algorithm for our project is "Decision Tree"

In [142]:
''''
clf=KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train,y_train)
pred = clf.predict(X_train)
print(metrics.confusion_matrix(y_true = y_train, y_pred = pred))
predtest = clf.predict(X_test)
metrics.confusion_matrix(y_true = y_test, y_pred = predtest)
''''

SyntaxError: EOL while scanning string literal (Temp/ipykernel_13160/2943648095.py, line 8)